# Hair Detector

#### Created by: Matheus Gabriel

Project to detect type of hair using one deep learning model.


## Imports

In [2]:
import numpy as np
import tensorflow as tf
import os
import cv2
from keras.preprocessing import image
from os.path import isfile, join


## Load images to train and test the model

In [3]:
train_datagen = image.ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True )
train_set = train_datagen.flow_from_directory('data/Train/',
                                                 target_size = (80, 80),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 494 images belonging to 3 classes.


In [4]:
test_datagen = image.ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True )
test_set = test_datagen.flow_from_directory('data/Test/',
                                                 target_size = (80, 80),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 218 images belonging to 3 classes.


## Defining the layers of model

In [5]:
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[80, 80, 3]))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Dropout(0.3))

cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=128, activation=tf.keras.layers.LeakyReLU()))

cnn.add(tf.keras.layers.Dropout(0.3))

cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))

cnn.add(tf.keras.layers.Dropout(0.2))

cnn.add(tf.keras.layers.Dense(units=3, activation='softmax'))

## Training the model 

In [6]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

cnn.fit(x = train_set, validation_data = test_set, epochs = 30)

Epoch 1/30
16/16 [==============================] - 6s 366ms/step - loss: 1.0801 - accuracy: 0.4595 - val_loss: 1.1012 - val_accuracy: 0.3807
Epoch 2/30
16/16 [==============================] - 5s 337ms/step - loss: 1.0202 - accuracy: 0.5263 - val_loss: 1.0749 - val_accuracy: 0.4128
Epoch 3/30
16/16 [==============================] - 6s 368ms/step - loss: 0.9740 - accuracy: 0.5486 - val_loss: 1.0274 - val_accuracy: 0.4587
Epoch 4/30
16/16 [==============================] - 6s 365ms/step - loss: 0.8970 - accuracy: 0.5850 - val_loss: 1.0861 - val_accuracy: 0.4450
Epoch 5/30
16/16 [==============================] - 6s 362ms/step - loss: 0.8934 - accuracy: 0.5911 - val_loss: 1.0446 - val_accuracy: 0.5138
Epoch 6/30
16/16 [==============================] - 6s 368ms/step - loss: 0.8273 - accuracy: 0.6457 - val_loss: 1.0352 - val_accuracy: 0.5092
Epoch 7/30
16/16 [==============================] - 6s 391ms/step - loss: 0.8302 - accuracy: 0.6377 - val_loss: 1.0140 - val_accuracy: 0.5275
Epoch 

## Function to return the type of hair

In [7]:
def predictCabelo(img):
    test_image = image.load_img(img, target_size = (80, 80))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)

    if(result[0][0] == max(result[0])):
        return 'cacheado'
    
    if(result[0][1] == max(result[0])):
        return 'liso'

    if(result[0][2] == max(result[0])):
        return'ondulado'

## Load `advertising.mp4` to extract frames and generate the predictions

In [8]:
vidcap = cv2.VideoCapture('advertising.mp4')

def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(f"frames/{count}.jpg", image)
        prediction = predictCabelo(f"frames/{count}.jpg")
        cv2.rectangle(image,(150,15),(500,85),(255,255,255), -1)
        cv2.rectangle(image,(150,15),(500,85),(56,142,72), 2)
        cv2.putText(image,prediction,(0 + 200,0 + 0 + 50), cv2.FONT_HERSHEY_COMPLEX,1, (56,142,72), 2, cv2.LINE_AA)  
        cv2.imwrite(f"frames/{count}.jpg", image)
    return hasFrames

sec = 0
frameRate = 0.5
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

## Join the frames in `video.avi`

In [9]:
pathIn= './frames/'
pathOut = 'video.avi'
fps = 0.5
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

files.sort(key = lambda x: x[5:-4])
files.sort()
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

files.sort(key = lambda x: x[5:-4])
for i in range(len(files)):
    filename=pathIn + files[i]
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()